# Example of loop annotation in S.cerevisiae

## One step annotation

In [1]:
#import
import numpy as np
import LASCA_pipeline
import cooler

In [2]:
#import cool file
S=cooler.Cooler('Saccer_1000_no_norm.cool')

#set resolution from cool
resolution=S.info['bin-size']

#set maximum loop size
distance_bins=40 #40Kb

#list of chromosomes
chroms=S.chromnames

#remove chrM from data
chroms.remove(u'chrM')

In [3]:
#load HiC contacts
#raw count
mtx_raw_S=S.matrix(balance=False).fetch(u'chrI')
#Balanced
mtx_S=S.matrix(balance=True).fetch(u'chrI')


In [4]:
#Set up parameters and annotate
#set up end bin of matrix 
end_bin=mtx_S.shape[0]-1 

LASCA_pipeline.LASCA_processing(raw_mtx=mtx_raw_S,mtx=mtx_S,chr_name='chrI',output_bedpe_name='Example_chrI.bedpe',\
                 resolution=resolution,start_bin=0,end_bin=end_bin,distance_bins=distance_bins,
                 FDR=0.05,q=0.95,adjust_by_scale=False,min_cluster=3,filter_bins=2,q_value_trhd=0.05,Intensity=True,\
                 as_intervals=True,bin_coverage=0.25,save_qvalues_mtx=False,use_filter=False)

Calculating coverage per bin...
('Filter poor bins with coverage <=', 0.25)
Filtering complete.
Start fitting Weibull distribution...
Distribution fitted.
Calculate q-values...
Finished!
q-value matrix not saved
Finished!


## Building pipeline from functions

In [5]:
#load HiC contacts
#raw count
mtx_raw_S=S.matrix(balance=False).fetch(u'chrI')
#Balanced
mtx_S=S.matrix(balance=True).fetch(u'chrI')

In [6]:
#NaNs to zeros
np.nan_to_num(mtx_S,copy=False)
np.nan_to_num(mtx_raw_S,copy=False)
#set up end bin of matrix 
end_bin=mtx_S.shape[0]-1 

In [7]:
#calculate q-value at specific distance range
q_vals_full_chrom=LASCA_pipeline.Get_pvalue_v7(mtx_raw_S,mtx_S,resolution,0*resolution,end_bin*resolution,40*resolution)
#convert q-values to log10
result_full_chrom=LASCA_pipeline.convert_to_log10_square_matrix(q_vals_full_chrom)
#convert to square q-value matrix
qvals_mtx=LASCA_pipeline.Get_qvals_mtx_v2(result_full_chrom,distance_bins)
#cluster significant dots
tmp_df=LASCA_pipeline.cluster_dots(qvals_mtx,mtx_S,3,filter_dist=2,q_value_treshold=0.05)
#Coordinates of cluster centers
coord=LASCA_pipeline.Get_coordinates(tmp_df,Intensity=True,as_intervals=True)

Calculating coverage per bin...
('Filter poor bins with coverage <=', 0.25)
Filtering complete.
Start fitting Weibull distribution...
Distribution fitted.
Calculate q-values...
Finished!


In [8]:
LASCA_pipeline.Export_as_bedpe(coord,'chrI',resolution,'Example_chrI_2.bedpe')

'Saved: Example_chrI_2.bedpe'